# Improving Linear Classifiers

In [ ]:
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

## Processing the data 

In [ ]:
def read_data(corpus_file, test_size=0.2, random_state=0):
    X = []
    Y = []
    with open(corpus_file, encoding="utf-8") as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )

def create_pipeline(clf, select_k_best=1000):
    return make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=select_k_best),
        Normalizer(),        
        clf
    )

def measure_pipeline(pipeline):
    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print("Training time: {:.2f} sec.".format(t1 - t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print("Accuracy: {:.4f}.".format(accuracy_score(Ytest, Yguess)))

# LinearClassifier baseclass
### This is the class provided with the perceptron in the zipped file linked in the introduction

In [ ]:
class LinearClassifier(BaseEstimator):    
    def decision_function(self, X):        
        return X.dot(self.w)

    def predict(self, X):        
        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):  
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):       
        return np.array([1 if y == self.positive_class else -1 for y in Y])



# LinearSVC and LogisticRegression
### They follow the perceptron structure with some minor changes

We initialize t=1 to avoid the division by zero corner case during the first iteration. However this only applies to the models which use the rescaling factor a, because we use (1/a) during the calculation of the loss function, but to be able to benchmark every model with each other we stay consistent.

In [ ]:
class LinearSVC(LinearClassifier):
    def __init__(self, n_iter=20):
        self.n_iter = n_iter

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features        

        t = 1
        for i in range(self.n_iter):
            for x, y in zip(X, encoded_Y):
                t += 1
                learning_rate = 1 / (t * regularization)
                score = x.dot(self.w)

                self.w = (1 - learning_rate * regularization) * self.w

                if y * score < 1:
                    self.w += (learning_rate * y) * x

In [ ]:
class LogisticRegression(LinearClassifier):
    def __init__(self, n_iter=20):
        self.n_iter = n_iter

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features
        
        t = 1
        for i in range(self.n_iter):
            for x, y in zip(X, encoded_Y):
                t += 1
                learning_rate = 1 / (t * regularization)                           

                self.w = (1 - learning_rate * regularization) * self.w

                self.w += (y * x) / (1 + np.exp(y * (x.dot(self.w))))            


# Evaluation of LinearSVC and LogisticRegression

These are the times we compare the optimized classifiers with

In [ ]:
# Read and split two train and test set
Xtrain, Xtest, Ytrain, Ytest = read_data("all_sentiment_shuffled.txt")

# Set up the preprocessing steps and the classifier.
pipeSVC = create_pipeline(LinearSVC())
pipeLR = create_pipeline(LogisticRegression())

print("LinearSVC")
measure_pipeline(pipeSVC)
print("\nLogisticRegression")
measure_pipeline(pipeLR)

LinearSVC
Training time: 7.09 sec.
Accuracy: 0.8242.

LogisticRegression
Training time: 8.28 sec.
Accuracy: 0.8212.


### Here the same classifiers are run, but using stochastic gradient descent. This saves quite a lot of time and preserves accuracy

In [ ]:
class BatchLogisticRegression(LinearClassifier):
    def __init__(self, n_iter=20, frac=0.2):
        self.n_iter = n_iter
        self.frac = frac

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features
        
        t = 1
        n = int(n_instances*self.frac)
        
        for i in range(self.n_iter):
            for _ in range(n):
                idx = np.random.randint(0, high=n_instances)
                x = X[idx]
                y = encoded_Y[idx]
                t += 1
                learning_rate = 1 / (t * regularization)                           

                self.w = (1 - learning_rate * regularization) * self.w

                self.w += (y * x) / (1 + np.exp(y * (x.dot(self.w))))            


## Using random batches with uniform random selection for training pairs

In [ ]:
class BatchLinearSVC(LinearClassifier):
    def __init__(self, n_iter=20, frac=0.2):
        self.n_iter = n_iter
        self.frac = frac

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features

        t = 1
        n = int(n_instances*self.frac)
        for i in range(self.n_iter):
            for _ in range(n):
                idx = np.random.randint(0, high=n_instances)
                x = X[idx]
                y = encoded_Y[idx]
                t += 1
                learning_rate = 1 / (t * regularization)
                score = x.dot(self.w)

                self.w = (1 - learning_rate * regularization) * self.w

                if y * score < 1:
                    self.w += (learning_rate * y) * x

In [ ]:
# Set up the preprocessing steps and the classifier.
# frac here is the percentage of the number of instances used in the iterations
pipeSVC = create_pipeline(BatchLinearSVC(frac=0.4))
pipeLR = create_pipeline(BatchLogisticRegression(frac=0.4))

print("BatchLinearSVC")
measure_pipeline(pipeSVC)
print("\nBatchLogisticRegression")
measure_pipeline(pipeLR)

BatchLinearSVC
Training time: 4.84 sec.
Accuracy: 0.8258.

BatchLogisticRegression
Training time: 7.34 sec.
Accuracy: 0.8334.


# Optimizing LR
### (a) Faster linear algebra operations
Here the boost functions are implemented. This saves up to around 2 seconds.

In [ ]:
from scipy.linalg.blas import ddot, dscal, daxpy

# a)
class LinearSVC(LinearClassifier):
    def __init__(self, n_iter=20):
        self.n_iter = n_iter

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features        

        t = 1
        for i in range(self.n_iter):
            a = 1.0
            for x, y in zip(X, encoded_Y):
                t += 1
                learning_rate = 1 / (t * regularization)
                score = ddot(x,self.w)
                a = 1 - learning_rate * regularization
                dscal(a, self.w)                

                if y * score < 1:  
                    daxpy(x, self.w, a=learning_rate * y)             

# Test optimized LR
pipeSVC = create_pipeline(LinearSVC())
print("LinearSVC")
measure_pipeline(pipeSVC)

LinearSVC
Training time: 4.41 sec.
Accuracy: 0.8242.


### (b) Using sparse vectors
Here we used the sparse model used in the sparse perceptron as boilderplate and adapt it to work with LinearSVC

In [ ]:
def create_sparse_pipeline(clf, ngram_range=(1,1)):
    return make_pipeline(
        TfidfVectorizer(ngram_range=ngram_range),        
        Normalizer(),        
        clf
    )

def add_sparse_to_dense(x, w, factor):   
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    return np.dot(w[x.indices], x.data)


class SparseSVC(LinearClassifier):    
    def __init__(self, n_iter=20):        
        self.n_iter = n_iter

    def fit(self, X, Y):        
        self.find_classes(Y)

        encoded_Y = self.encode_outputs(Y)
        n_features = X.shape[1]
        self.w = np.zeros(X.shape[1])    
        
        # For increased iteration speed, saves
        XY = list(zip(X, encoded_Y))

        regularization = 1 / n_features    
        t = 1
        for i in range(self.n_iter):
            a = 1.0
            for x, y in XY:
                t += 1
                learning_rate = 1 / (t * regularization)
                score = sparse_dense_dot(x, self.w)
                a = 1 - learning_rate * regularization
                self.w *= a                

                if y * score < 1:  
                    add_sparse_to_dense(x, self.w, learning_rate * y)

In [ ]:
# Test optimized LR
pipeSVC = create_sparse_pipeline(SparseSVC())
print("SparseSVC ngram_range=(1,1)")
measure_pipeline(pipeSVC)

pipeSVC = create_sparse_pipeline(SparseSVC(), ngram_range=(1,2))
print("\nSparseSVC ngram_range=(1,2)")
measure_pipeline(pipeSVC)

# When ngram_range is (1,2) the normal LinearSVC crashes due to lack of ram.
# Meanwhile the sparse SVC takes quite a long time for large data, but it works.
pipeSVC = create_sparse_pipeline(LinearSVC())
print("\nLinearSVC ngram_range=(1,1)")
measure_pipeline(pipeSVC)

SparseSVC ngram_range=(1,1)
Training time: 13.67 sec.
Accuracy: 0.8334.

SparseSVC ngram_range=(1,2)
Training time: 90.90 sec.
Accuracy: 0.8670.

LinearSVC ngram_range=(1,1)
Training time: 37.61 sec.
Accuracy: 0.8334.


We can see that the sparse model is quite much faster than the standard model for ngram_range=(1,1). For ngram_range=(1,2) as mentioned, the standard model crashes due to lack of ram.

### (c) Speeding up the scaling operation

Here the rescaling of w is done once instead of every iteration, as shown in the clarification paper in section 2.4. This does not really matter in the normal case, but in the sparse case it save several seconds.

In [ ]:
class RescalingLinearSVC(LinearClassifier):
    def __init__(self, n_iter=20):
        self.n_iter = n_iter

    def fit(self, X, Y):
        self.find_classes(Y)
        encoded_Y = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        n_instances, n_features = X.shape
        self.w = np.zeros(n_features)
        regularization = 1 / n_features        

        t = 1
        a = 1.0
        for i in range(self.n_iter):            
            for x, y in zip(X, encoded_Y):
                t += 1
                learning_rate = 1 / (t * regularization)
                
                score = ddot(x,self.w) * a
                a = (1 - learning_rate * regularization) * a
                
                
                if y * score < 1:                    
                    daxpy((x*learning_rate*y), self.w, a=1/a)                    

        dscal(a, self.w)               

# Test optimized LR
pipeSVC = create_pipeline(RescalingLinearSVC())
print("LinearSVC")
measure_pipeline(pipeSVC)

LinearSVC
Training time: 5.62 sec.
Accuracy: 0.8242.


In [ ]:
class SparseRescalingSVC(LinearClassifier):    
    def __init__(self, n_iter=20):        
        self.n_iter = n_iter

    def fit(self, X, Y):        
        self.find_classes(Y)

        encoded_Y = self.encode_outputs(Y)
        n_features = X.shape[1]
        self.w = np.zeros(X.shape[1])    
        
        # For increased iteration speed, saves
        XY = list(zip(X, encoded_Y))

        regularization = 1 / n_features    
        t = 1
        a = 1.0
        for i in range(self.n_iter):
            for x, y in XY:
                t += 1
                learning_rate = 1 / (t * regularization)
                
                score = sparse_dense_dot(x, self.w) * a   
                a = (1 - learning_rate * regularization) * a

               

                if y * score < 1:  
                    add_sparse_to_dense(x, self.w, learning_rate * y / a)
        self.w *= a


pipeSVC = create_sparse_pipeline(SparseRescalingSVC())
print("RescalingSVC")
measure_pipeline(pipeSVC)

RescalingSVC
Training time: 7.70 sec.
Accuracy: 0.8334.
